In [88]:
import pandas as pd
import numpy as np
from sympy.combinatorics import Permutation
import subprocess

p = 'santa-2023/'
path = pd.read_csv(p + 'puzzles.csv')
info = pd.read_csv(p + 'puzzle_info.csv')
sub = pd.read_csv(p+'sample_submission.csv')

info['allowed_moves_count'] = info['allowed_moves'].map(lambda x: {k: Permutation(v) for k, v in eval(x).items()})
paths = pd.merge(path, info, how='left', on='puzzle_type')
paths = pd.merge(paths, sub, how='left', on='id')

In [89]:
def do_action(state: list[str], action_id: str, allowed_moves: dict[str, list[int]]):
    s=state.copy()
    if action_id[0]=="-":
        action = allowed_moves[action_id[1:]]
        for i,a in enumerate(action):
            s[a]=state[i]
    else:
        action = allowed_moves[action_id]
        for i,a in enumerate(action):
            s[i]=state[a]
    return s

def simulate(state: str, actions: str, allowed_moves: str):
    s=state.split(";")
    allowed=eval(allowed_moves)
    for a in actions.split("."):
        s = do_action(s, a, allowed)
    return ";".join(s)

def simulate_inverse(state: str, actions: str, allowed_moves: str):
    s=state.split(";")
    allowed=eval(allowed_moves)
    for a in reversed(actions.split(".")):
        if a[0]=="-":
            s = do_action(s, a[1:], allowed)
        else:
            s = do_action(s, f"-{a}", allowed)
    return ";".join(s)

In [90]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master

# git clone https://github.com/dwalton76/rubiks-cube-NxNxN-solver.git
# cd rubiks-cube-NxNxN-solver
# make init
# pip install kociemba

In [112]:
# N=2
# N=3
# N=4
N=5
# N=6
# N=7
# N=8
# N=9
# N=10
# N=19
# N=33

NN=N*N

In [113]:
# org = "ABCDEF"
# import itertools
# org
# p = itertools.permutations(org)
# for a in p:
#     dct=dict(zip(a, "URFDLB"))
#     print(dct)
    
#     row=paths.loc[130]
#     id = row['id']
#     cur_state = row['initial_state']
#     print(cur_state)
#     input_state=state2ubl(cur_state)
#     print(id, input_state)

#     result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
#     output=result.stdout.decode()
#     if not output.startswith("Solution: "):
#         print("Fail!", output, result.stderr.decode())
#         continue
#     sol=output.replace("Solution: ","").replace("\n","")
#     print(sol)
#     print("find!!")
#     break

In [114]:
# "ACBDEF"
# "URFDLB"
# s="ABCDEF"
# s[0] + s[2] + s[1] + s[5] + s[4] + s[3]
dct=dict(zip("ACBFED", "URFDLB"))
dct

{'A': 'U', 'C': 'R', 'B': 'F', 'F': 'D', 'E': 'L', 'D': 'B'}

In [115]:
def state2ubl(state):

    # dct = {}
    # for i,u in enumerate(U):
    #     dct[state.split(';')[4+i*NN]] = u
    # print(dct)
    s = ''.join([dct[f] for f in state.split(';')])
    
    return s[:NN] + s[2*NN:3*NN] + s[NN:2*NN] + s[5*NN:] + s[4*NN:5*NN] + s[3*NN:4*NN]

In [116]:
eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0]).keys()

dict_keys(['f0', 'f1', 'f2', 'f3', 'f4', 'r0', 'r1', 'r2', 'r3', 'r4', 'd0', 'd1', 'd2', 'd3', 'd4'])

In [117]:
moves = eval(info.loc[info['puzzle_type'] == f'cube_{N}/{N}/{N}']['allowed_moves'].values[0])
for move in list(moves):
    moves['-'+move] = np.argsort(moves[move]).tolist()

# TODO: 本質的に使わない操作が存在する？
# 他を回すと自分を固定する
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver/tree/master/rubikscubennnsolver
M = {}
M["U"] = f'-d{N-1}'
M["R"] = "r0"
M["B"] = f"-f{N-1}"
M["F"] = "f0"
M["L"] = f"-r{N-1}"
M["D"] = "d0"
# if N>=4:
M["Uw"] = f'-d{N-1}.-d{N-2}'
M["Rw"] = "r0.r1"
M["Bw"] = f"-f{N-1}.-f{N-2}"
M["Fw"] = "f0.f1"
M["Lw"] = f"-r{N-1}.-r{N-2}"
M["Dw"] = "d0.d1"
# if N>=6:
#     M["2Uw"] = f'-d{N-1}.-d{N-2}'
#     M["2Rw"] = "r0.r1"
#     M["2Bw"] = f"-f{N-1}.-f{N-2}"
#     M["2Fw"] = "f0.f1"
#     M["2Lw"] = f"-r{N-1}.-r{N-2}"
#     M["2Dw"] = "d0.d1"
#     M["3Uw"] = f'-d{N-1}.-d{N-2}.-d{N-3}'
#     M["3Rw"] = "r0.r1.r2"
#     M["3Bw"] = f"-f{N-1}.-f{N-2}.-f{N-3}"
#     M["3Fw"] = "f0.f1.f2"
#     M["3Lw"] = f"-r{N-1}.-r{N-2}.-r{N-3}"
#     M["3Dw"] = "d0.d1.d2"
for i in range(2, N):
    M[f"{i}Uw"] = ".".join([f"-d{N-1-k}" for k in range(i)])
    M[f"{i}Rw"] = ".".join([f"r{k}" for k in range(i)])
    M[f"{i}Bw"] = ".".join([f"-f{N-1-k}" for k in range(i)])
    M[f"{i}Fw"] = ".".join([f"f{k}" for k in range(i)])
    M[f"{i}Lw"] = ".".join([f"-r{N-1-k}" for k in range(i)])
    M[f"{i}Dw"] = ".".join([f"d{k}" for k in range(i)])
    
for m in list(M):
    M[m+"2"] = M[m] + '.' + M[m]
    if "-" in M[m]:
        M[m+"'"] = M[m].replace("-","")
    else:
        # M[m+"'"] = "-"+M[m]
        M[m+"'"] = "-"+".-".join(M[m].split("."))
M

{'U': '-d4',
 'R': 'r0',
 'B': '-f4',
 'F': 'f0',
 'L': '-r4',
 'D': 'd0',
 'Uw': '-d4.-d3',
 'Rw': 'r0.r1',
 'Bw': '-f4.-f3',
 'Fw': 'f0.f1',
 'Lw': '-r4.-r3',
 'Dw': 'd0.d1',
 '2Uw': '-d4.-d3',
 '2Rw': 'r0.r1',
 '2Bw': '-f4.-f3',
 '2Fw': 'f0.f1',
 '2Lw': '-r4.-r3',
 '2Dw': 'd0.d1',
 '3Uw': '-d4.-d3.-d2',
 '3Rw': 'r0.r1.r2',
 '3Bw': '-f4.-f3.-f2',
 '3Fw': 'f0.f1.f2',
 '3Lw': '-r4.-r3.-r2',
 '3Dw': 'd0.d1.d2',
 '4Uw': '-d4.-d3.-d2.-d1',
 '4Rw': 'r0.r1.r2.r3',
 '4Bw': '-f4.-f3.-f2.-f1',
 '4Fw': 'f0.f1.f2.f3',
 '4Lw': '-r4.-r3.-r2.-r1',
 '4Dw': 'd0.d1.d2.d3',
 'U2': '-d4.-d4',
 "U'": 'd4',
 'R2': 'r0.r0',
 "R'": '-r0',
 'B2': '-f4.-f4',
 "B'": 'f4',
 'F2': 'f0.f0',
 "F'": '-f0',
 'L2': '-r4.-r4',
 "L'": 'r4',
 'D2': 'd0.d0',
 "D'": '-d0',
 'Uw2': '-d4.-d3.-d4.-d3',
 "Uw'": 'd4.d3',
 'Rw2': 'r0.r1.r0.r1',
 "Rw'": '-r0.-r1',
 'Bw2': '-f4.-f3.-f4.-f3',
 "Bw'": 'f4.f3',
 'Fw2': 'f0.f1.f0.f1',
 "Fw'": '-f0.-f1',
 'Lw2': '-r4.-r3.-r4.-r3',
 "Lw'": 'r4.r3',
 'Dw2': 'd0.d1.d0.d1',
 "Dw'": '-d0.-

In [118]:
# paths.loc[paths['puzzle_type'] == 'cube_3/3/3'].head()
# paths.loc[paths['puzzle_type'] == 'cube_4/4/4'].head()
# paths.loc[paths['puzzle_type'] == 'cube_5/5/5'].head()
# paths.loc[paths['puzzle_type'] == 'cube_6/6/6'].head()
# paths.loc[paths['puzzle_type'] == 'cube_7/7/7'].head()
# paths.loc[paths['puzzle_type'] == 'cube_33/33/33'].head()

In [98]:
# http://aym.pekori.to/rubik/pattern-01.html
input_state=simulate_inverse(
    paths.loc[30, "solution_state"], ## 普通の解
    paths.loc[130, "moves"],
    paths.loc[130, "allowed_moves"],
)
# paths.loc[130, "initial_state"]
input_state=state2ubl(input_state)
input_state

'RUBUFFDBULLULUUDDDFRBURDUBFFLLRBFDFLFBRBDRBDLRLUFLDBRR'

In [652]:
i=0

row=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[i]
id = row['id']
cur_state = row['initial_state']
print(cur_state)
input_state=state2ubl(cur_state)
print(id, input_state)

A;D;D;B;E;B;B;E;D;F;C;B;F;F;F;F;E;D;A;D;F;A;A;D;E;C;D;E;C;B;C;B;C;B;C;F;D;B;C;B;B;E;E;A;B;E;E;A;D;B;A;A;D;E;A;A;F;E;F;E;F;E;E;D;E;E;F;A;A;A;C;F;C;A;D;D;A;C;E;B;F;C;D;E;F;D;C;D;F;A;E;C;B;F;D;C;D;B;C;E;C;C;B;C;D;B;B;C;D;A;C;A;C;B;E;F;E;B;C;A;F;F;B;B;F;B;B;D;F;F;C;D;A;D;C;A;F;A;F;F;B;C;A;A;E;D;D;E;E;A
210 UBBFLFFLBDRFDDDDLBUBDUUBLUUBLUUDLDLDLLBLLDUUURDRUBRBLRFRFRFRDBFRFFLLUFLLUBFFFBDDRBUBRUDUDDFRUULBBLLURRFRBFFRBURURFLDLFRUDDFFDBURLFDRBLDBRBDULRFDBRBFRL


In [653]:
# input_state=state2ubl(paths.loc[130, "solution_state"])
# input_state

In [638]:
result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
output=result.stdout.decode()
if not output.startswith("Solution: "):
    print("Fail!", output, result.stderr.decode())
sol=output.replace("Solution: ","").replace("\n","")
sol

"D U' B' L B' U2 R U' L F U L2 U2 R2 F2 U R2 B2 U F2"

In [639]:
mmoves = '.'.join([M[m] for m in sol.split(' ')])
mmoves

'd0.d2.f2.-r2.f2.-d2.-d2.r0.d2.-r2.f0.-d2.-r2.-r2.-d2.-d2.r0.r0.f0.f0.-d2.r0.r0.-f2.-f2.-d2.f0.f0'

In [640]:
# simulate_inverse(
simulate(
    paths.loc[130, "initial_state"], ## 普通の解
    mmoves,
    paths.loc[130, "allowed_moves"],
)

'C;E;B;A;B;C;E;C;D;A;B;F;D;C;A;F;F;A;E;F;E;C;A;D;D;C;B;F;F;B;A;E;B;A;F;D;F;E;B;D;F;E;A;B;D;C;B;C;E;D;D;E;A;C'

In [641]:
I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
    if simulate(
        paths.loc[130, "initial_state"],
        mmoves+("."+init_moves if init_moves else ""),
        paths.loc[130, "allowed_moves"],
    )==paths.loc[130, "solution_state"]:
        print(mmoves+"."+init_moves)

In [451]:
# https://github.com/dwalton76/rubiks-cube-NxNxN-solver?tab=readme-ov-file
# sol="D U' B' L B' U2 R U' L F U L2 U2 R2 F2 U R2 B2 U F2"
# sol="D' L2 Bw2 Rw' D Fw' D' Rw2 U Bw2 D B L Dw2 Rw' L2 U' F D2 Dw2 Rw2 F' L2 D' Bw2 Lw2 D2 F2 R2 D' Lw2 U L D B2 L' D L B' D2 F' R2 B2 D R2 U2 R2 D2 B2 L2 D'"

In [452]:
new_state = cur_state
mmoves = '.'.join([M[m] for m in sol.split(' ')])

for move in mmoves.split('.'):
    new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))

# I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
    temp_state = new_state
    if len(init_moves) > 0:
        for move in init_moves.split('.'):
            temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))

    if temp_state == row['solution_state']:
        print(f'solved id: {id}')
        if len(init_moves) > 0:
            mmoves += '.' + init_moves
        if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
            print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
            paths.iloc[id,7] = mmoves
        break


solved id: 277
improved: new length 15067 vs current length 24419


# まとめて解く

In [662]:
import subprocess

for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    id = row['id']
    cur_state = row['initial_state']
    problem=state2ubl(cur_state)
    print("-"*30)
    print(id, problem)

    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {problem}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
        continue
    sol=output.replace("Solution: ","").replace("\n","")
    print("solved", sol)

    new_state = cur_state
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    for move in mmoves.split('.'):
        new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
    
    I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        temp_state = new_state
        if len(init_moves) > 0:
            for move in init_moves.split('.'):
                temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
    
        if temp_state == row['solution_state']:
            print(f'solved id: {id}')
            if len(init_moves) > 0:
                mmoves += '.' + init_moves
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves
            break


------------------------------
150 FUUURFRDURUBDBLURDLRUULRFLLFRUBLBRUFDUFRUFDRRBUDDLRBDFLFLBDDLLBFDDBLLDBRDBRLUDFFBBFLFBUBLDRFURFB
solved D' L2 Bw2 Rw' D Fw' D' Rw2 U Bw2 D B L Dw2 Rw' L2 U' F D2 Dw2 Rw2 F' L2 D' Bw2 Lw2 D2 F2 R2 D' Lw2 U L D B2 L' D L B' D2 F' R2 B2 D R2 U2 R2 D2 B2 L2 D'
solved id: 150
improved: new length 98 vs current length 700
------------------------------
151 DUURLBFBFBFLLUBFUBRFLRRULUDRDURRBBDRBUBFULDBDDFBBLDRFFDRDDRDFFDBLDUUFULRFRULUBLLDLBFFLBDULFRURRL
solved Rw' Bw Rw B2 Dw' R2 F' Dw2 Rw' B2 U Rw' U2 Rw' Uw2 F L2 U' Bw2 U' F Dw2 B2 Bw2 D' F2 Fw2 B2 Lw2 R2 D Rw2 L2 D' L F L2 F L' F B' R U2 B2 U L2 F2 D2 R2 F2 U L2
solved id: 151
improved: new length 102 vs current length 610
------------------------------
152 RRDBDUBLUDBLFDUUFFDURFLLDLUBLLFBRBLLRLDDRRFFUBUDLLFFFURDUDLULRRRFFRDUFBBBBRUBBLBRRBUURUBDFDLDFFD
solved Dw2 Rw' D R2 Uw R' Fw' F' B' Rw' Bw2 Rw' Dw2 B' L' U Rw' Rw2 D B' Uw2 D B Dw2 B2 L2 D2 R2 B2 D' F2 Fw2 U' B2 Lw2 U2 D R B' U' B' U D2 R2 F R U' R2 U 

KeyboardInterrupt: 

In [467]:
paths[["id", "moves"]].to_csv("solver_submit.csv", index=None)

# ストライプ

In [119]:
import subprocess

normal_state=paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iloc[0]["solution_state"]
for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    if row["solution_state"]==normal_state:
        continue
    id = row['id']
    cur_state = row['initial_state']
    print("-"*30)
    print(id)

    input_state=simulate_inverse(
        normal_state, ## 普通の解
        row["moves"],
        row["allowed_moves"],
    )
    input_state=state2ubl(input_state)
    
    result=subprocess.run(f"python ./rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state {input_state}", shell=True, capture_output=True)
    output=result.stdout.decode()
    if not output.startswith("Solution: "):
        print("Fail!", output, result.stderr.decode())
    sol=output.replace("Solution: ","").replace("\n","")
    print("sol", sol)
    mmoves = '.'.join([M[m] for m in sol.split(' ')])
    
    I = [".".join([f"r{i}"for i in range(N)]), ".".join([f"d{i}"for i in range(N)]), ".".join([f"f{i}"for i in range(N)])]
    for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
        if simulate(
            row["initial_state"],
            mmoves+("."+init_moves if init_moves else ""),
            row["allowed_moves"],
        )==row["solution_state"]:
            print(mmoves+"."+init_moves)
            if len(paths.iloc[id,7].split('.')) > len(mmoves.split('.')):
                print(f"improved: new length {len(mmoves.split('.'))} vs current length {len(paths.iloc[id,7].split('.'))}")
                paths.iloc[id,7] = mmoves+("."+init_moves if init_moves else "")
            break


------------------------------
235
sol Uw' Bw L' Uw2 Bw' Lw2 Fw2 Dw Lw B Uw' U' Bw2 R' Uw2 R2 Fw' Uw2 L Bw' D2 Uw2 B U2 Rw2 Fw2 L' F' L D' Fw2 R' D2 L D2 Fw2 D2 R Fw2 L2 Dw2 B2 R' Uw2 Bw2 D R2 D' Rw2 U2 Bw2 Lw2 F2 Fw2 Rw2 D' L2 Rw2 D' Lw2 D2 U' R D B U' R D2 R' F' R' B' R2 B2 U2 F2 U2 B2 U L2 F2 U
d4.d3.-f4.-f3.r4.-d4.-d3.-d4.-d3.f4.f3.-r4.-r3.-r4.-r3.f0.f1.f0.f1.d0.d1.-r4.-r3.-f4.d4.d3.d4.-f4.-f3.-f4.-f3.-r0.-d4.-d3.-d4.-d3.r0.r0.-f0.-f1.-d4.-d3.-d4.-d3.-r4.f4.f3.d0.d0.-d4.-d3.-d4.-d3.-f4.-d4.-d4.r0.r1.r0.r1.f0.f1.f0.f1.r4.-f0.-r4.-d0.f0.f1.f0.f1.-r0.d0.d0.-r4.d0.d0.f0.f1.f0.f1.d0.d0.r0.f0.f1.f0.f1.-r4.-r4.d0.d1.d0.d1.-f4.-f4.-r0.-d4.-d3.-d4.-d3.-f4.-f3.-f4.-f3.d0.r0.r0.-d0.r0.r1.r0.r1.-d4.-d4.-f4.-f3.-f4.-f3.-r4.-r3.-r4.-r3.f0.f0.f0.f1.f0.f1.r0.r1.r0.r1.-d0.-r4.-r4.r0.r1.r0.r1.-d0.-r4.-r3.-r4.-r3.d0.d0.d4.r0.d0.-f4.d4.r0.d0.d0.-r0.-f0.-r0.f4.r0.r0.-f4.-f4.-d4.-d4.f0.f0.-d4.-d4.-f4.-f4.-d4.-r4.-r4.f0.f0.-d4.r0.r1.r2.r3.r4.r0.r1.r2.r3.r4.d0.d1.d2.d3.d4
improved: new length 178 vs curre

In [120]:
paths[["id", "moves"]].to_csv("solver_submit_stripe.csv", index=None)

In [121]:
for i, row in paths.loc[paths['puzzle_type'] == f'cube_{N}/{N}/{N}'].iterrows():
    assert simulate(
        row["initial_state"],
        row["moves"],
        row["allowed_moves"]
    )==row["solution_state"]